In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Intro

Welcome to the BlendingToolKit (BTK) quickstart! This tutorial will guide you through the basic functionalities of BTK. If you are new to BTK, we recommend you to go through this notebook before moving on to the other tutorials.

We will show you how to generate a catalog of galaxy blends, draw images of these blends, use algorithms implemented within BTK to deblend these images, and finally use metrics within BTK to assess the performance of the deblender.

# Setup

First, we will need to import the relevant python packages and modules:

In [ ]:
%matplotlib inline
import numpy as np
from tqdm import tqdm 

In [ ]:
import btk
import btk.survey
import btk.draw_blends
import btk.catalog
import btk.sampling_functions

# Drawing Blends

Before generating images of blends, we require some setup. 

**First**, we need to specify a catalog of galaxies to draw. We will use a subset of the CATSIM catalog, which contains `100` galaxies with realistic parameters.

BTK uses a wrapper class (e.g. `btk.catalog.CatsimCatalog`) to store information about the galaxy catalog you pass in. You can easily import the CATSIM catalog from a FITS file using the `from_file` method as demonstrated here with our example catalog:

In [ ]:
catalog_name = "../data/input_catalog.fits" # contains ~85k entries
catalog = btk.catalog.CatsimCatalog.from_file(catalog_name)
catalog.table[:5] # display 5 first entries of table containing the actual catalog information.

**Second**, we require specifying a *sampling function*. The sampling function is a custom class within BTK that is used to determine which galaxies are drawn for each blend and what their locations within the blend should be. 

Specifically, we implement this functionality with the `SamplingFunction` class, which is a `callable` like a function. It takes as argument the astropy table contained within the `Catalog` object above, selects galaxies to be drawn in a given blend, and returns their (possibly modified) entries. Usually the `ra` and `dec` columns of these entries are modified to be relative to the center of the postage stamp containing the blend. 

For this tutorial, we will use the *default sampling function*, this function is implemented in the `btk.sampling_functions` module and is called `DefaultSampling`. This function draws a random number of galaxies from the catalog, uniformly distributed between 1 and `max_number`, and places them randomly within the postage stamp with some maximum shift `max_shift` w.r.t the center of the postage stamp.

The `DefaultSampling` function can be instantiated as follows:

In [ ]:
stamp_size = 24.0  # Size of the stamp, in arcseconds
max_number = 3    # Maximum number of galaxies in a blend
max_shift = 1.0    # Maximum shift of the galaxies, in arcseconds
seed = 0 # random seed for reproducibility purposes # seed = 8
sampling_function = btk.sampling_functions.DefaultSampling(
    max_number=max_number, min_number=max_number,  # always get `max_number` galaxies
    stamp_size=stamp_size, max_shift=max_shift, 
    min_mag = 24, max_mag = 25,
    seed = seed)

Here is an example output of the `DefaultSampling` function:

In [ ]:
blend_catalog = sampling_function(catalog.table)
blend_catalog # ra and dec are now relative to the center of the blend (not the original ones)

**Third**, we need to specify a *survey*. This will determine the pixel scale, sky level, and PSF of the images we will draw. 

BTK relies on the [galcheat](https://github.com/aboucaud/galcheat) package, which contains several `galcheat.survey.Survey` instances, which store the parameters for different surveys (including LSST, HSC, HST COSMOS...). The parameters represent physical parameters of the survey (mirror size, pixel scale). Each survey also contains several `galcheat.filter.Filter` objects with the parameters specific to each filter (exposure time, zeropoint). 

In BTK, we use the `btk.survey.Survey` class to store the survey parameters. This class is a simple wrapper around the `galcheat.survey.Survey` class, and contains additional information such as the PSF model.

For this tutorial, we will import the survey corresponding to LSST.

In [ ]:
LSST = btk.survey.get_surveys("LSST")

You can see the parameters of the survey by printing the `Survey` object:

In [ ]:
print(LSST)

**Finally**, now that we have finished our setup, we can create the `DrawBlendsGenerator`. This object is a python `generator`, meaning it can be called with `next(generator)` to generate a new batch of blend catalogs and images. 

The `DrawBlendsGenerator` takes as arguments the `Catalog` object, the `SamplingFunction` object, and the `Survey` object. It also takes a few other arguments, such as the number of images to draw per batch, the size of the postage stamps, and the number of galaxies to draw per blend.

In [ ]:
batch_size = 100

draw_generator = btk.draw_blends.CatsimGenerator(
    catalog,
    sampling_function,
    LSST,
    batch_size=batch_size,
    stamp_size=stamp_size,
    njobs=1,
    add_noise="all",
    seed=seed, # use same seed here
)

# generate batch 100 blend catalogs and images.
blend_batch = next(draw_generator)

The output of the `DrawBlendsGenerator` is a class called `BlendBatch`, which contains the truth catalogs, simulated blends, and isolated images (among other metadata). 

For example to access the images of blends, we can use the following syntax:

In [ ]:
blend_batch.blend_images.shape 
# shape = (batch_size, n_bands, stamp_size, stamp_size)

The truth catalogs are contained in the `catalog_list` object which is a list of catalogs, one per each blend in the batch. In total there are `batch_size` blends in the batch.

In [ ]:
blend_batch.catalog_list[0] 
# blend_list is a list of astropy tables, one for each blend in the batch.

There is also additional information which you can inspect by printing the `BlendBatch` object. Specifically, the PSF and WCS. The PSF is a list of `galsim.GSObject` objects, one per filter, and the WCS is an astropy `WCS` object.

In [ ]:
blend_batch

We can proceed to display a given blend using `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt

# plot the first blend in the batch, with the r-band
plt.imshow(blend_batch.blend_images[0, 2, :, :], cmap="gray")

# plot centers
plt.scatter(blend_batch.catalog_list[0]["x_peak"], 
            blend_batch.catalog_list[0]["y_peak"], c="r", marker="x")

We can use the function from `astropy` to plot in RGB (see [documentation](https://docs.astropy.org/en/stable/api/astropy.visualization.make_lupton_rgb.html) here for more info):

In [ ]:
from astropy.visualization import make_lupton_rgb

im = blend_batch.blend_images[0]
bands = [1, 2, 3] # g, r, i 
stretch = np.max(im) - np.min(im)
Q = 0.1

img_rgb = make_lupton_rgb(im[bands[0]], im[bands[1]], im[bands[2]], stretch=stretch, Q=Q)
plt.imshow(img_rgb)

Or our own BTK utility function which uses a linear stretch:

In [ ]:
from btk.plotting import get_rgb

rgb = get_rgb(im[bands])
plt.imshow(rgb)

# Deblending

The next big part of BTK is the deblending. We will use the `Deblender` class to deblend the images we just drew. By 'deblending' we mean that we want to recover the individual galaxies that were blended together in the images. This includes recovering their positions, fluxes, and shapes.

In this tutorial we will use the `SExtractor` deblender, which is implemented in the `btk.deblender` module. Specifically the class `SepSingleband` is a wrapper around the [SEP](https://sep.readthedocs.io/en/v1.1.x/) package, which is a python implementation of the [SExtractor](https://www.astromatic.net/software/sextractor) algorithm that performs detection on a single band.

Here is how we create the `Deblender` object:

In [ ]:
from btk.deblend import SepSingleBand

In [ ]:
deblender = SepSingleBand(max_n_sources=3, # same as above
                          thresh=3, # threshold pixel value for detection (see SEP docs)
                          use_band=2 # measure on 'r' band
                          )

You can use the `Deblender` object to deblend the images we drew earlier. The `Deblender` object takes as input the `BlendBatch` object, and returns a new `DeblenderBatch` object with the deblended images and/or catalogs.

In [ ]:
output = deblender.deblend(0, blend_batch) # deblend the first blend in the batch.

Let's inspect the output which is a `DeblendedExample` object. The output contains at least a `catalog` attribute, which in this case contains the locations for the deblended objects in the first blend of the batch.

In [ ]:
print(output)

In [ ]:
output.catalog

There is also an option to run the deblender on the full batch with the `__call__` method of any `Deblender` object. This method takes as input the `BlendBatch` object, and returns a new `DeblenderBatch` object with the deblended images and catalogs for each blend in the batch.

In [ ]:
deblend_batch = deblender(blend_batch, njobs=1) # deblend the whole batch

You can access them as attributes of the `DeblenderBatch` object:

In [ ]:
deblend_batch.catalog_list[0] # predicted catalog for the first blend in the batch

Let's compare the results from both the truth catalog and predicted catalog.

In [ ]:
import matplotlib.pyplot as plt

# plot the first blend in the batch, with the r-band
plt.imshow(blend_batch.blend_images[0, 2, :, :], cmap="gray")

# plot centers of truth
plt.scatter(blend_batch.catalog_list[0]["x_peak"], blend_batch.catalog_list[0]["y_peak"], c="r", marker="x")

# plot centers of prediction

# need to use wcs to convert ra and dec to x and y
ra, dec = deblend_batch.catalog_list[0]["ra"], deblend_batch.catalog_list[0]["dec"]
x, y = blend_batch.wcs.world_to_pixel_values(ra / 3600, dec / 3600) # need to convert to arcseconds
plt.scatter(x, y, c="b", marker="+", s=50)

We can also inspect the deblended images and compare with the isolated ones. 

**Note:** Images do not necessarily line up in general as below, need to match (see next section)

In [ ]:
# plot isolated images from truth 
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

clip = 40 # zoom into images

true_images = blend_batch.isolated_images[0, :, 2, clip:-clip, clip:-clip]
pred_images = deblend_batch.deblended_images[0, :, clip:-clip, clip:-clip] # only contains r-band

# use global vmin and vmax
vmin = 0
vmax = 9000

for ii in range(3):
    ax = axes.flatten()[ii]
    ax.imshow(blend_batch.isolated_images[0, ii, 2, clip:-clip, clip:-clip], cmap="gray", 
              vmin=vmin, vmax=vmax)

# plot isolated images from prediction
for ii in range(3, 6):
    ax = axes.flatten()[ii]
    ax.imshow(deblend_batch.deblended_images[0, ii-3, 0, clip:-clip, clip:-clip], cmap="gray", 
              vmin=vmin, vmax=vmax)

# add colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(axes[0, 0].imshow(blend_batch.isolated_images[0, 0, 2, clip:-clip, clip:-clip], cmap="gray", vmin=vmin, vmax=vmax), cax=cbar_ax)

# Matching

To be able to compare the predictions of each deblender to the truth or to other deblenders we need to match the predicted catalogs to the truth catalogs. This is done using the `Matcher` class.

We have implemented several matching algorithms in the `btk.match` module. For this tutorial we will use the `PixelHungarianMatcher` matcher, which uses the Hungarian algorithm to perform the matching.

In [ ]:
from btk.match import PixelHungarianMatcher

In [ ]:
matcher = PixelHungarianMatcher(pixel_max_sep=5.0 # maximum separation to consider match, in pixels
)

Before matching, we need to add pixel centroids to the catalogs of `deblend_batch`. As it currently only contains `ra` and `dec`. We can do it easily with the utility function `btk.utils.add_pixel_columns`:

In [ ]:
deblend_batch.catalog_list[0].colnames # no x_peak, y_peak

In [ ]:
from btk.utils import add_pixel_columns
wcs = blend_batch.wcs
add_pixel_columns(deblend_batch.catalog_list, wcs)
deblend_batch.catalog_list[0].colnames # now has x_peak, y_peak

In [ ]:
# matchers operate on `catalog_lists`, so we need to extract them from our batch classes. 
true_catalog_list = blend_batch.catalog_list
pred_catalog_list = deblend_batch.catalog_list
matching = matcher(true_catalog_list, pred_catalog_list) # matching object

The matching object contains useful information (for a given batch...): 

In [ ]:
(matching.true_matches[0], # index of each true source that is matched with predicted 
 matching.pred_matches[0], #index of each pred source that is matched with truth
   matching.n_true[0],  # number of truth total
   matching.n_pred[0],  # number of predicted total 
   matching.detected[0], # boolean array for each true source if it was matched or not, 
   matching.matched[0] # boolean array for each pred source if it was matched or not, 
)

We can already do something useful, like compute average separation between matched predictions on the batch (in pixels)

In [ ]:
# first we match the catalogs so that all objects between catalogs are aligned
matched_true_catalogs = matching.match_true_catalogs(true_catalog_list)
matched_pred_catalogs = matching.match_pred_catalogs(pred_catalog_list)

In [ ]:
# now compute distances on each catalog and average
dist = [] # list of distances between matched objects
for true_cat, pred_cat in zip(matched_true_catalogs, matched_pred_catalogs):
    if len(true_cat) == 0 or len(pred_cat) == 0:
        continue
    x1, y1 = true_cat['x_peak'], true_cat['y_peak']
    x2, y2 = pred_cat['x_peak'], pred_cat['y_peak']
    dist.append(np.sqrt( (x1-x2)**2 + (y1-y2)**2 ))
# unpack list of distances into a single array
dist = np.concatenate(dist)
np.mean(dist)

# Metrics

Now we show how to compute metrics, metrics in BTK are separated into three classes: **detection**, **segmentation**, and **reconstruction**. For this tutorial we will focus on computing 'recall', which is type of detection metric.

Metrics in BTK are designed to work on batches, every time a new batch is computed the metric update its state to accumulate the results of each batch. Creating a metric object to start accumulating results is simple, just pass in the `batch_size`:

## Detection

In [ ]:
from btk.metrics.detection import Recall

In [ ]:
recall = Recall(batch_size)

The detection type metrics specifically take in two arrays `detected` and `matched`, which are boolean arrays indicating if a true and predicted object (respectively) were matched (see matching above). 
Let's compute precision for the `matching` object we computed above.

In [ ]:
detected = matching.detected
matched = matching.matched

recall(detected, matched)

We can accumulate recalls over multiple batches to get a more accurate results (for our specific draw blend generator):

In [ ]:
# get generator w/o bar for nicer output
draw_generator = btk.draw_blends.CatsimGenerator(
    catalog,
    sampling_function,
    LSST,
    batch_size=batch_size,
    stamp_size=stamp_size,
    njobs=1,
    add_noise="all",
    seed=seed, # use same seed here
    use_bar=False
)

In [ ]:
n_batches = 10
for n in tqdm(range(n_batches)):
    blend_batch = next(draw_generator)
    deblend_batch = deblender(blend_batch, njobs=1)
    true_catalog_list = blend_batch.catalog_list
    pred_catalog_list = deblend_batch.catalog_list
    add_pixel_columns(deblend_batch.catalog_list, wcs)
    matching = matcher(true_catalog_list, pred_catalog_list)
    detected = matching.detected
    matched = matching.matched
    recall(detected, matched) # accumulate values internally

Multiple metrics can be computed for each batch to save time. For simplicity, just focusing on one metric.

Let's look at all the recall values obtained for each batch:

In [ ]:
print(recall.values)

And we can compute the aggragate, which for this metric is just the mean of the recall values: 

In [ ]:
recall.aggregate()

## Reconstruction

In [ ]:
from btk.metrics.reconstruction import MSE
mse = MSE(batch_size)

In [ ]:
# for reconstruction and segmentation, need to match first and then apply metric
iso_images1 = blend_batch.isolated_images[:, :, 2] # only r-band
iso_images2 = deblend_batch.deblended_images[:, :, 0] 
iso_images_matched1 = matching.match_true_arrays(iso_images1)
iso_images_matched2 = matching.match_pred_arrays(iso_images2)

In [ ]:
mse(iso_images_matched1, iso_images_matched2) # automatically accounts for empty arrays

# Measurement

Another useful quantity we can obtain are what in BTK we call 'measurements'. These are quantities that can be derived from galaxy images like aperture fluxes, ellipticities, or signal-to-noise ratio that might help you understand the performance of your algorithm.

BTK provides functions to easily compute these quantities from batches. For example, for SNR we have a `get_snr` function. It takes in the `isolated_images` inside `blend_batch` and the `sky_level` or background that can be easily computed. 

In [ ]:
from galcheat.utilities import mean_sky_level
from btk.measure import get_snr

# we will continue using 'r' band
sky_level = mean_sky_level(LSST, 'r').to_value('electron') # gain = 1
iso_images = blend_batch.isolated_images[:,:, 2] # only r-band
snr = get_snr(iso_images, sky_level)
snr.shape # one for each source in each batch

Histogram of SNRs for one batch: 

In [ ]:
plt.hist(snr.flatten(), histtype='step')

We can also compare true ellipticity with deblended ellipticity (measured with KSB) easily for a single batch: 

In [ ]:
from btk.measure import get_ksb_ellipticity

#obtaining images
iso_images1 = blend_batch.isolated_images[:,:, 2] # only r-band
psf_r =blend_batch.psf[2] # psf in r-band
iso_images2 = deblend_batch.deblended_images[:, :, 0] # select band

#matching
iso_images_matched1 = matching.match_true_arrays(iso_images1)
iso_images_matched2 = matching.match_pred_arrays(iso_images2)

#compute ellipticity
ellips1 = get_ksb_ellipticity(iso_images_matched1, psf_r, pixel_scale=0.2)

# NOTE: assumed deblended images are psf convolved with same psf
ellips2 = get_ksb_ellipticity(iso_images_matched2, psf_r, pixel_scale=0.2) 


# mask nan's (non-matches), look at first component only
e11 = ellips1[:, :, 0].flatten()
e12 = ellips2[:, :, 0].flatten()

mask = (~np.isnan(e12)) & (np.abs(e12) < 1) & (~np.isnan(e11))
e11 = e11[mask]
e12 = e12[mask]

In [ ]:
# look at first component 
plt.scatter(e11, e12)
plt.plot([-1, 1], [-1, 1])